In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# 필요 함수들
def extract(df: pd.DataFrame, name: str, newname: str, regex: str):
    df[newname]=df[name].apply(lambda x:re.search(regex,str(x)).group(1) if re.search(regex,str(x)) else '')

def fixmajor(df: pd.DataFrame, major: str):
    return df[(df['학부(과)']==major) | (df['학부(과)'].isna())].reset_index(drop = True)

def sortout(string: str):
    return {'요일':re.search('(^[월화수목금토일])', string).group(1), '교시':re.search('^[월화수목금토일](.*?)\[', string).group(1), '동':re.search('\[0*(\d+)\-', string).group(1), '호':re.search('\-(\d+)\]', string).group(1)}

<>:9: SyntaxWarning: invalid escape sequence '\['
<>:9: SyntaxWarning: invalid escape sequence '\['
<>:9: SyntaxWarning: invalid escape sequence '\-'
<>:9: SyntaxWarning: invalid escape sequence '\['
<>:9: SyntaxWarning: invalid escape sequence '\['
<>:9: SyntaxWarning: invalid escape sequence '\-'
/tmp/ipykernel_1186/2437312824.py:9: SyntaxWarning: invalid escape sequence '\['
  return {'요일':re.search('(^[월화수목금토일])', string).group(1), '교시':re.search('^[월화수목금토일](.*?)\[', string).group(1), '동':re.search('\[0*(\d+)\-', string).group(1), '호':re.search('\-(\d+)\]', string).group(1)}
/tmp/ipykernel_1186/2437312824.py:9: SyntaxWarning: invalid escape sequence '\['
  return {'요일':re.search('(^[월화수목금토일])', string).group(1), '교시':re.search('^[월화수목금토일](.*?)\[', string).group(1), '동':re.search('\[0*(\d+)\-', string).group(1), '호':re.search('\-(\d+)\]', string).group(1)}
/tmp/ipykernel_1186/2437312824.py:9: SyntaxWarning: invalid escape sequence '\-'
  return {'요일':re.search('(^[월화수목금토일])', string

In [31]:
def timecheck(ttbl: pd.DataFrame, info: str):
    return ttbl.loc[np.array(sortout(info)['교시'].split(',')).astype(int), sortout(info)['요일']]

def insert(df: pd.DataFrame, ttbl: pd.DataFrame, index: int):
    ttbl.loc[np.array(sortout(df.loc[index, '강의정보1'])['교시'].split(',')).astype(int), sortout(df.loc[index, '강의정보1'])['요일']] = index
    if not pd.isnull(df.loc[index, '강의정보2']):
        ttbl.loc[np.array(sortout(df.loc[index, '강의정보2'])['교시'].split(',')).astype(int), sortout(df.loc[index, '강의정보2'])['요일']] = index

def getinfo(df: pd.DataFrame, index: int):
    result = [df.loc[index, '강의정보1']]
    if not pd.isnull(df.loc[index, '강의정보2']):
        result.append(df.loc[index, '강의정보2'])
    return result

In [14]:
costmin

,1,2,3,3-1,4,5,6,11,21,22,...,451,452,453,454,455,456,459,501,505,601
1,0,4,7,6,8,5,9,5,20,19,...,10,8,9,10,9,11,11,18,10,15
2,4,0,6,5,7,8,12,7,7,8,...,7,5,7,9,7,9,9,9,8,14
3,8,6,0,4,2,12,16,11,11,11,...,7,4,7,6,8,5,5,10,7,9
3-1,6,5,4,0,5,11,14,10,9,9,...,5,2,5,7,5,6,6,8,7,12
4,9,7,2,5,0,13,17,12,12,11,...,8,4,8,6,8,4,5,10,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,12,10,5,6,4,16,20,15,15,11,...,9,5,9,6,8,0,4,10,7,9
459,12,10,5,7,6,17,20,15,19,8,...,7,5,7,3,5,4,0,7,4,12
501,11,9,9,8,10,14,22,14,15,4,...,6,7,5,4,4,9,6,0,3,17
505,10,9,7,8,8,14,17,14,16,5,...,5,7,5,2,3,7,4,4,0,15


In [34]:
getinfo(df, 4)

['수3,4,5[351-0115]']

In [49]:
def filter_by_time(df: pd.DataFrame, ttbl: pd.DataFrame, index: int):
    if (timecheck(ttbl, df.loc[index, '강의정보1']) != -1).any():
        return False
    if not pd.isnull(df.loc[index, '강의정보2']):
        return (timecheck(ttbl, df.loc[index, '강의정보2']) == -1).all()
    return True

def filter_by_costmin(df: pd.DataFrame, ttbl: pd.DataFrame, costmin: pd.DataFrame, index: int):
    li = ttbl[ttbl != -1].stack().unique().astype(int)
    for i in li:
        for j in getinfo(df, i):
            # res.append({'번호':i, '요일':sortout(j)['요일'], '교시':sortout(j)['교시'][len(sortout(j)['교시'])]+1, '동':sortout(j)['동'], '이동시간':costmin.loc[sortout(j)['동'], sortout(index)['동']]})
            for k in getinfo(df, index):
                if sortout(j)['요일'] == sortout(k)['요일'] and str(int(sortout(j)['교시'].split(',')[len(sortout(j)['교시'])])+1) in sortout(k)['교시'].split(',') and costmin.loc[sortout(j)['동'], sortout(k)['동']] > 9:
                    return False
    return True

In [5]:
# 자료 불러오기
costmin = pd.read_csv('costmin.csv', index_col=0)
df0 = pd.read_csv('전공교양_가좌대면.csv')

# costmin.csv와 전공교양_가좌대면.csv 필요

In [8]:
df0

,순번,이수구분,교과목명,캠퍼스구분,학점,강의시간/강의실,강의정보1,강의정보2,학부(과),영역구분
0,5,전필,국문학개론,가좌캠퍼스,3.0,"금1,2,3[101-0352]","금1,2,3[101-0352]",NaN,국어국문학과,NaN
1,6,전선,한국언어·문학·문화현장답사,가좌캠퍼스,1.0,수4[101-0348],수4[101-0348],NaN,국어국문학과,NaN
2,7,전선,문학문화비평,가좌캠퍼스,3.0,"월8,9,10[101-0348]","월8,9,10[101-0348]",NaN,국어국문학과,NaN
3,8,전선,한국어음운론,가좌캠퍼스,3.0,"월1,2[101-0348],수3[101-0348]","월1,2[101-0348]",수3[101-0348],국어국문학과,NaN
4,9,전선,시창작론,가좌캠퍼스,3.0,"화8,9[101-0348],목9[101-0348]","화8,9[101-0348]",목9[101-0348],국어국문학과,NaN
...,...,...,...,...,...,...,...,...,...,...
3354,1010,일반선택,성공취업과전략,가좌캠퍼스,2.0,"수8,9[024-0258]","수8,9[024-0258]",NaN,NaN,NaN
3355,1011,일반선택,직무분석과취업,가좌캠퍼스,2.0,"수6,7[024-0243]","수6,7[024-0243]",NaN,NaN,NaN
3356,1012,일반선택,군사안보학,가좌캠퍼스,3.0,"수1,2[033-T101],금6,7,8,9[033-T101]","수1,2[033-T101]","금6,7,8,9[033-T101]",NaN,NaN
3357,1013,일반선택,군사조직리더십,가좌캠퍼스,3.0,"수1,2[033-T201],금6,7,8,9[033-T201]","수1,2[033-T201]","금6,7,8,9[033-T201]",NaN,NaN


In [7]:
# 전공 선택
df = fixmajor(df0, '수학물리학부')

In [12]:
# 시간표 만들기 & 시간표 초기화
num = 15 # num교시까지
ttbl = pd.DataFrame(np.full((num,7), -1))
ttbl.columns = '월 화 수 목 금 토 일'.split(' ')
ttbl.index = range(num+1)[1:]
ttbl = ttbl.astype(int)

In [38]:
# 시간표에 과목 추가
if filter_by_time(df, ttbl, 44) and filter_by_costmin(df, ttbl, costmin, 44): # 시간표가 겹치거나 이동시간이 10분 이상인지 검사
    insert(df,ttbl,44)
if filter_by_time(df, ttbl, 123) and filter_by_costmin(df, ttbl, costmin, 123):
    insert(df,ttbl,123)
if filter_by_time(df, ttbl, 12) and filter_by_costmin(df, ttbl, costmin, 12):
    insert(df,ttbl,333)
ttbl

,월,화,수,목,금,토,일
1,11,-1,-1,-1,-1,-1,-1
2,11,-1,-1,-1,-1,-1,-1
3,11,-1,-1,-1,44,-1,-1
4,-1,-1,-1,-1,44,-1,-1
5,-1,-1,-1,-1,-1,-1,-1
6,123,12,-1,-1,-1,-1,-1
7,123,12,-1,-1,-1,-1,-1
8,-1,12,-1,-1,-1,-1,-1
9,-1,12,-1,-1,-1,-1,-1
10,-1,-1,-1,-1,-1,-1,-1


In [43]:
df.index

RangeIndex(start=0, stop=978, step=1)

In [47]:
# 최종
def check(df, ttbl, costmin):
    return [i for i in df.index if filter_by_time(df, ttbl, i) and filter_by_costmin(df, ttbl, costmin, i)]

In [50]:
check(df, ttbl, costmin)

IndexError: list index out of range